# Folium : 지도 활용
: 세계지도를 기본 지원하고 다양한 스타일의 지도 이미지를 제공한다

In [3]:
!pip install folium

In [4]:
import folium
import pandas as pd

In [8]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)
seoul_map

In [9]:
# 지도를 HTML 파일로 저장하기
seoul_map.save("../Data/seoul.html")

### 지도 스타일 적용

In [10]:
seoul_map2 = folium.Map(
    location=[37.55,126.98], 
    zoom_start=12,
    tiles = 'Stamen Terrain'
    )
seoul_map2

In [11]:
seoul_map3 = folium.Map(
    location=[37.55,126.98], 
    zoom_start=12,
    tiles = 'Stamen Toner'
    )
seoul_map3

---
# 지도에 마커 표시하기
- 서울 시내 주요 대학교의 위치를 표시

In [14]:
df = pd.read_excel("../Data/서울지역 대학교 위치.xlsx")
df.columns = ['학교명','위도','경도']
df.head()

,학교명,위도,경도
0,KAIST 서울캠퍼스,37.592573,127.046737
1,KC대학교,37.548345,126.854797
2,가톨릭대학교(성신교정),37.585922,127.004328
3,가톨릭대학교(성의교정),37.499623,127.006065
4,감리교신학대학교,37.567645,126.961610


In [25]:
seoul_map = folium.Map(
    location=[37.55,126.98], 
    zoom_start=13,
    #tiles = 'Stamen Terrain'
    )
# 대학교 위치 정보를 Marker로 표시
for name,lat,lng in zip(df.학교명,df.위도,df.경도):
    folium.Marker([lat,lng],popup=name).add_to(seoul_map)
seoul_map

---
# 지도에 원형 마커 표시


In [36]:
seoul_map = folium.Map(
    location=[37.55,126.98], 
    zoom_start=13,
    #tiles = 'Stamen Terrain'
    )
# 대학교 위치 정보를 CircleMarker로 표시
for name,lat,lng in zip(df.학교명,df.위도,df.경도):
    folium.CircleMarker(
        [lat,lng],
        popup=name,
        color = 'purple',
        radius = 10, # 원의 반지름값을 바꿔서 줄 수 있다~~~~
        fill_color = 'coral',
        fill_opacity = 0.7,
        ).add_to(seoul_map)
seoul_map

---
### 지도 영역에 단계구분포(Choropleth Map) 표시하기
: 행정구역과 같이 지도상의 어떤 경계에 둘러싸인 영역에 색을 칠하거나 음영등의 정보를 나타내는 시각화 방법

In [37]:
import json

In [61]:
# 경기도 인구변화 데이터를 불러와서 데이터프레임으로 변환 
df = pd.read_excel("../Data/경기도인구데이터.xlsx", index_col="구분")
df.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
구분,,,,,,,,,,,
수원시장안구,287474,285803,290798,293692,290999,291757,300908,301196,299016,296479,293828
수원시권선구,310553,308177,304097,306783,321176,332633,331773,339835,351053,358393,365653
수원시팔달구,216945,213235,219833,216503,209772,206794,204805,203479,199180,198515,193311
수원시영통구,252730,260210,258421,260557,266542,289074,310671,329718,335375,340654,349836
성남시수정구,256744,248452,242161,241070,236123,233997,228167,223539,219531,232841,236932


In [63]:
df.columns = df.columns.map(str)

In [64]:
# 경기도 시군구 경계정보를 가진 geo-json 파일 불러오기
geo_path = "../Data/경기도행정구역경계.json"
try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig'))

geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': 31380,
    'name': '양평군',
    'name_eng': 'Yangpyeong-gun',
    'base_year': 2013},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.56113535909492, 37.6256560539258],
      [127.57290963929243, 37.61087369350498],
      [127.56366973908277, 37.5842624297699],
      [127.52226606559191, 37.5828628697518],
      [127.50048767007603, 37.569036373178626],
      [127.47687473909174, 37.574448241913856],
      [127.47565866803647, 37.60512112601634],
      [127.44699730711024, 37.64497584741164],
      [127.4272349102434, 37.66284420898682],
      [127.40156700708162, 37.64806970784708],
      [127.3732642199939, 37.6453987554284],
      [127.3542931884407, 37.6250006376975],
      [127.34360057873045, 37.588997440929354],
      [127.31002843450217, 37.53523876142835],
      [127.30923294884336, 37.5135706079458],
      [127.32809611134748, 37.53133849534739],
      [127.3663244453597, 37.5

In [68]:
# 경기도 지도 만들기
g_map = folium.Map(
    location=[37.5502, 126.982],
    tiles = "Stamen Terrain",
    zoom_start = 9
)

year = '2017'

# Choropleth 클래스로 단계구분 표시하기
folium.Choropleth(
    geo_data = geo_data, # 경기 지도 경계 데이터
    data = df[year], # 표시하려는 데이터
    columns = [df.index, df[year]], # 열 지정
    fill_color = 'YlOrRd', #
    fill_opacity = 0.7, #
    line_opacity = 0.3, #
    threshold_scale = [10000, 100000, 300000, 500000,700000 ] , # 색을 줄 범위를 정함
    key_on = 'feature.properties.name' # json에서 name 의 위치 (feature밑에 , properties 밑에 , name에 있다)
).add_to(g_map)
g_map.save("../Data/gyonggi_population_"+ year +".html")
g_map